In [1]:
import sys
sys.path.append("../utils")
from spark_functions import create_SparkSession
from logging_functions import inject_logging
from string_functions import to_snake_case
spark = create_SparkSession()

In [3]:
import os
os.listdir('../data/')

['Camera_Traffic_Counts_20250125.csv',
 'Radar_Traffic_Counts_20250125.csv',
 'Traffic_Detectors_20250125.csv']

In [2]:
base_path = "../data"


camera_traffic = spark.read.option("header",True).csv(f"{base_path}/Camera_Traffic_Counts_20250125.csv")
for col_ in camera_traffic.columns:
    camera_traffic = camera_traffic.withColumnRenamed(col_,to_snake_case(col_))

radar_traffic = spark.read.option("header",True).csv(f"{base_path}/Radar_Traffic_Counts_20250125.csv")
for col_ in radar_traffic.columns:
    radar_traffic = radar_traffic.withColumnRenamed(col_,to_snake_case(col_))

traffic_detectors = spark.read.option("header",True).csv(f"{base_path}/Traffic_Detectors_20250125.csv")
for col_ in traffic_detectors.columns:
    traffic_detectors = traffic_detectors.withColumnRenamed(col_,to_snake_case(col_))

['record_id',
 'atd_device_id',
 'read_date',
 'intersection_name',
 'direction',
 'movement',
 'heavy_vehicle',
 'volume',
 'speed_average_miles_per_hour',
 'speed_std_dev',
 'seconds_in_zone_average',
 'seconds_in_zone_std_dev',
 'month',
 'day',
 'year',
 'hour',
 'minute',
 'day_of_week',
 'bin_duration_seconds']

In [ ]:
['Record ID',
 'ATD Device ID',
 'Read Date',
 'Intersection Name',
 'Direction',
 'Movement',
 'Heavy Vehicle',
 'Volume',
 'Speed Average (Miles Per Hour)',
 'Speed StdDev',
 'Seconds in Zone Average',
 'Seconds in Zone StdDev',
 'Month',
 'Day',
 'Year',
 'Hour',
 'Minute',
 'Day of Week',
 'Bin Duration (Seconds)']

In [ ]:
['record _i_d',
 'a_t_d _device _i_d',
 'read _date',
 'intersection _name',
 'direction',
 'movement',
 'heavy _vehicle',
 'volume',
 'speed _average (_miles _per _hour)',
 'speed _std_dev',
 'seconds in _zone _average',
 'seconds in _zone _std_dev',
 'month',
 'day',
 'year',
 'hour',
 'minute',
 'day of _week',
 'bin _duration (_seconds)']

In [11]:
spark.stop()